# 1.1 Daum News Locator
> 목적 : "키워드, 언론사, 기간"을 받아서,  
=> '다음 뉴스' 검색을 통해, 해당 기사의 url을 긁어오는 리스트를 가져온다. 
  
  
> 경로 :  
=> 다음을 실행 후 => 타겟 키워드 검색 
  => 뉴스 탭으로 입장 => 기간 + 언론사 필터 설정  
  => 해당 기사의 url을 받아오는 리스트 생성

### 1.1 필요 패키지 Import

In [1]:
# 이 작업을 수행하기 위해 꼭 필요한 패키지들입니다. 아래 패키지들이 install되어있는지 체크해주세요.
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install bs4
# !pip install selenium

In [1]:
import pandas as pd
import numpy as np
import datetime  # < 'datetime' 데이터 형태를 조작하기 편하도록 해줍니다. * 참고 pandas.datetime

import platform
import matplotlib.pyplot as plt

import re     #파이썬에서 정규표현식(Regular Expression) 지원 모듈

%matplotlib inline # < matpliotlib 을 주피터 노트북에서 사용할 경우 plot들을 결과 창 안에 맞춰 출력시켜주도록 해줍니다. 
# 설정하지 않아도 크게 문제가 되지는 않지만, matpliot을 쓸 때 custumary하게 주로 지정해주고 시작합니다. 

# matplot 한글 폰트 인식 코드 
# matpliot 자체에서는 한글폰트를 자동으로 인식하지 못합니다. 
# matpliot의 plotting paremeter에서 직접 한글폰트 경로를 넣어주면 plot에 한글 출력이 가능해집니다. 
path = "c:/Windows/Fonts/malgun.ttf" # < 윈도우에 '맑은 고딕' 폰트의 경로입니다.
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':  # < 맥킨토시 사용자의 경우 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # < 윈도우 사용자의 경우 
    font_name = font_manager.FontProperties(fname=path).get_name() 
    rc('font', family=font_name)
    # 사용하고자 하는 한글 폰트의 경로를 path에 지정해주세요.
else:
    print('Unknown system')    

plt.rcParams['axes.unicode_minus'] = False

UsageError: unrecognized arguments: # < matpliotlib 을 주피터 노트북에서 사용할 경우 plot들을 결과 창 안에 맞춰 출력시켜주도록 해줍니다.


* 파이썬에서 웹 크롤링을 돕는 도구 1. BeeautifulSoup와 2. Selenium을 import합니다. 
    
: 두 패키지는 서로 독립적으로 웹 크롤링을 수행하고, 각각의 기능이 조금씩 장단점이 존재합니다.   

   
저의 경우, Selenium을 주로 사용하고, selenium에서 제한적인 명령을 수행하기위해,   
필요에따라 BeautifulSoup를 사용했습니다. 
   

* Selenium은 클릭과 같은 활동형 명령이 상당히 편합니다.    
하지만 각 element(이하 el)의 부모 el, 형제 el, 자식 el 등   
이웃하는 el들을 지정해주기 조금 어렵습니다.   

* BeutifulSoup는 화면 내 모든 el들의 정보를 한번에 parsing한 후에    
각 원소 간의 이웃 관계를 따라 흘러들어가는 것에 장점이 있습니다.   
하지만 한 번에 현재 페이지를 parsing하는 방법 때문에,   
페이지를 클릭하면서 넘어다니는 것이 복잡한 문제가 있습니다. 

In [ ]:
# 파이썬에서 웹 크롤링을 돕는 도구 BeeautifulSoup와 Selenium을 import합니다. 
from bs4 import BeautifulSoup 

import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


* Headless모드 :   
* 참고 :  https://beomi.github.io/2017/09/28/HowToMakeWebCrawler-Headless-Chrome/ (나만의 웹 크롤러 만들기(7): 창없는 크롬으로 크롤링하기)   

웹 크롤링을 수행하면, 지정된 웹 탐색 작업을 자동으로 수행하는 동안 default로 크롬, 파이어폭스(프로그래머가 지정한 브라우저) 창이 열립니다.   

하지만 코드 에러를 잘 정리한 후에는, 다른 작업을 위해서라도 굳이 계속 화면이 켜지고 꺼지는게 반복 되도록 할 필요가 없기 때문에, 창이 '물리적으로' 열리지 않지만 동일한 웹 탐색 작업을 하도록 설정할 수 있습니다. 이렇게 창이 뜨지 않도록 하는 경우를 Headless mode라고 합니다. 

In [ ]:
### Headless Mode로 스크래핑 할 경우를 위한 option set-up
options = webdriver.ChromeOptions()
options.add_argument('headless') # < headless를 지정해주고
options.add_argument('window-size=1920x1080') # < 물리적으로 창이 열리지 않기 때문에, 코드를 만들때 쓰던 해상도와 혹시라도 달라져서 에러가 생기는 것을 방지하기 위해 브라우저의 해상도를 아예 지정해줍니다. 
options.add_argument("disable-gpu")

### 1.2 필수 입력 변수 : driver경로, 키워드, 언론사, 검색기간

* '다음 뉴스'를 거쳐, 원하는 언론사 별, 기간 별, 키워드 별 모든 기사의 URL을 가져오는 것이 목적입니다.   
  
  이 과정을 수행하기 위해 필요한 필수 매개변수들을 지정해줍니다. 

* 아래에는 예를들어, '공천' 키워드에 대해, '조선일보'에서 발간한, '19년 12월'한 달간의 기사를 모두 긁어오고자 한다고 가정해보겠습니다. 


In [2]:

# selenium은 Browser의 버전에 따른 Driver 사전 경로를 지정해 주어야 합니다. 
Chrome_directory = '../Driver/chromedriver' # < 파일 서버에 넣어놓은 파일은 크롬 v79.에 해당하는 driver 입니다. 
# Chrome의 현재 버전에 맞는 Driver를 먼저 다운로드 후, driver의 경로를 설정해주세요. 
# * 브라우저 업데이트가 자주 일어나기 때문에, 현재 가지고 있는 드라이버가 out-dated 되기 쉽습니다. 
# 에러가 날 경우, 브라우저의 버전을 확인해주시고, 브라우저 해당버전에 맞는 driver를 검색하여 새로 받아주세요. 

Target_keyword = '공천' # 키워드를 넣습니다. 예를들어 '공천','20대 국회'
journal_target = '조선일보' # 원하는 언론사를 지정합니다. 예를들어 '조선일보','한겨레','한국일보'

# 전체 목표 기간 설정 < 에러가 나지 않도록 str(YYYY-mm-dd)형식으로 지정해주세요.
total_start_date = '2019-12-09' # < 월요일 날짜로 지정해주세요
total_end_date = '2019-12-29' # < 일요일 날짜로 지정해주세요

# pandas.date_range(시작,종료, 리턴빈도).strftime(리턴날짜형식='').tolist()
Total_Weekly_Monday = pd.date_range(total_start_date,total_end_date, freq = 'W-MON').strftime(date_format = '%Y.%m.%d').tolist()
Total_Weekly_Sunday = pd.date_range(total_start_date, total_end_date, freq = 'W').strftime(date_format = '%Y.%m.%d').tolist()

print(Total_Weekly_Monday,' < 검색기간에 시작 날짜가 됩니다.')
print(Total_Weekly_Sunday,' < 검색기간에 종료 날짜가 됩니다.')
# 위에 값을 받아서 => for 반복문을 통해 기간 내 '주차별 검색'을 수행합니다.
# period_start_date = Total_Weekly_Monday[search_week_idx]
# period_end_date = Total_Weekly_Sunday[search_week_idx]






['2019.12.09', '2019.12.16', '2019.12.23']  < 검색기간에 시작 날짜가 됩니다.
['2019.12.15', '2019.12.22', '2019.12.29']  < 검색기간에 종료 날짜가 됩니다.


## 1.3 코드 실행 

In [5]:
for search_week_idx in range(len(Total_Weekly_Monday)):
    # for 반복문을 통해, 기간 내 '주차별 검색'을 수행합니다.
    period_start_date = Total_Weekly_Monday[search_week_idx]
    period_end_date = Total_Weekly_Sunday[search_week_idx]
    print('전체 ',len(Total_Weekly_Monday),'개 weeks      ----->    ',str(search_week_idx + 1),'번째 week \n')

    # 크롬드라이버 import
    Chrome_directory = 'C:/Users/seoun/Desktop/Python/Driver/chromedriver' 
    driver = webdriver.Chrome(Chrome_directory)
    # driver 지정을 늘 새로 > "driver = webdriver.Chrome" 을 항상 같이 해야 url오픈에 에러가 안남 

    # 크롬드라이버를 통해 Daum News open
    driver.get("https://www.daum.net/")
    driver.implicitly_wait(10) # 페이지 이동 중에 로드(상태)가 다 될 때까지 최장 10초를 대기

    ### 검색어 입력
    # SearchBox keyword 입력
    # Target_keyword = '20대 국회'  # < 위에서 입력값을 받습니다.
    driver.find_element_by_css_selector('input#q.tf_keyword').clear() 
    driver.find_element_by_css_selector('input#q.tf_keyword').send_keys(Target_keyword) 

    # '검색' 클릭
    driver.find_element_by_css_selector('button.ico_pctop.btn_search').send_keys(Keys.ENTER) # '검색'클릭
    # 페이지가 넘어가기 때문에 '원하는 작업'을 수행하기위한 wait의 default 값을 지정해줍니다.
    wait = WebDriverWait(driver, 60) # 완료되기를 기대하는 '지정된 작업'에 대해 Max 60초를 기다립니다.작업이 완료되지 않으면 Timeout에러가 발생합니다. 

    # 페이지가 넘어가기 때문에 '암묵적 대기'를 걸어줍니다.
    driver.implicitly_wait(10) # 페이지 이동 중에 로드(상태)가 다 될 때까지 최장 10초를 대기

    # 카테고리 > '뉴스' 클릭
    # 검색 카테고리는 검색어에 따라 그 위치가 달라집니다. 
    # Xpath로 '뉴스'클릭을 지정할 경우 ...div[인덱스숫자].. 에 인덱스 숫자가 3또는 4로 달라지면서
    # 다른 카테고리로 유입될 수 있습니다. > 따라서, 카테고리 탭 안에서 현재 '뉴스'탭의 위치를 찾고 
    # 그 후 클릭을 수행합니다. 

    cat_tab_elements = driver.find_element_by_css_selector('ul.gnb_search').find_elements_by_css_selector('span.txt_tab')
    for cat_idx in range(len(cat_tab_elements)):
        if cat_tab_elements[cat_idx].text == '뉴스':
            cat_tab_elements[cat_idx].click()
            break #  참고 * )
    # * : '뉴스'탭을 클릭한 후에는, 나머지 탭에 대한 경로가 유효하지 않지만, for는 끝나지 않았기 떄문에
    # StaleElementReferenceException 에러가 발생하게 됩니다. break로 반복문이 안전하게 Exit할 수 있도록 지정해줍니다. 

    driver.implicitly_wait(10) # 페이지 이동 중에 로드가 다 될 때 까지 최장 10초 대기

    period_more_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[2]/div[1]/a/span[2]'''
    try: # 로딩이 채 끝나기전 .click()이 수행되면 에러가 발생합니다. > 따라서 클릭이 가능할 때 까지 wait를 수행합니다.
        element = wait.until(EC.element_to_be_clickable((By.XPATH, period_more_xpath)))
        driver.find_element_by_xpath(period_more_xpath).click() # '기간 더보기 화살표'클릭
    except:
        print('기사 검색 기간 선택에 에러 발생')
        print('period_start_date :',period_start_date, 'period_end_date : ',period_start_date)

    period_start_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[2]/div[1]/div/div/div[1]/input'''
    period_end_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[2]/div[1]/div/div/div[2]/input'''
    apply_btn_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[2]/div[1]/div/div/button'''

    # 검색 시작 기간 입력
    driver.find_element_by_xpath(period_start_xpath).clear() 
    driver.find_element_by_xpath(period_start_xpath).send_keys(period_start_date)

    # 검색 종료 기간 입력
    driver.find_element_by_xpath(period_end_xpath).clear() 
    driver.find_element_by_xpath(period_end_xpath).send_keys(period_end_date)

    driver.find_element_by_xpath(apply_btn_xpath).send_keys(Keys.ENTER) 
    #'적용'버튼에 직접적으로 'enter'키 입력 (for unclickable 해결)

    driver.implicitly_wait(10) # 페이지 이동 중에 로드가 다 될 때 까지 최장 10초 대기

    ### 언론사 설정 
    journal_more_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[2]/div[2]/a/span[2]'''
    driver.find_element_by_xpath(journal_more_xpath).click() # '언론사 더보기'클릭

    journal_input_xpath='''/html/body/article/article/div/article/div/div[2]/div[1]/div[2]/div[2]/div/ul/li[2]/a'''
    driver.find_element_by_xpath(journal_input_xpath).click() # '언론사 직접입력'클릭

    journal_target_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[2]/div[2]/div/ul/li[2]/div/div[1]/input'''
    journal_apply_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[2]/div[2]/div/ul/li[2]/div/button'''

    # 타겟 언론사 입력
    driver.find_element_by_xpath(journal_target_xpath).clear() 
    driver.find_element_by_xpath(journal_target_xpath).send_keys(journal_target)

    driver.find_element_by_xpath(journal_apply_xpath).send_keys(Keys.ENTER)
    #'적용'버튼에 직접적으로 'enter'키 입력 (for unclickable 해결)

    element = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'wrap_cont')))
    time.sleep(1)
    # 기사가 다 로드 될 때 까지 최장 60초를 기다린다. 

    ### 스크래핑 할 모든 기사들의 URL 주소를 하나의 리스트에 긁어옵니다.
    posts_href_list = [] # <<< 전체 url을 담을 리스트
    paging_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[4]'''
    # 페이지 바가 있는 경우
    if (driver.find_element_by_xpath(paging_xpath).get_attribute('class')) == 'paging_comm':
        i = 0 
        while i == 0: 
            time.sleep(1)
            posts_href_list_in_page=[] # 임시리스트를 비우고
            # 현재 페이지의 전체 기사 URL을 찾아온다.
            every_posts_info_xpath='''/html/body/article/article/div/article/div/div[2]/div[1]/div[3]'''
            every_posts_info = driver.find_element_by_xpath(every_posts_info_xpath)
            every_href_list = every_posts_info.find_elements_by_css_selector('li') # 기사 하나 당 블록
            each_post_href_css = 'div.wrap_cont > div > div.wrap_tit.mg_tit > a'
            posts_href_list_in_page = list(map(lambda x: x.find_element_by_css_selector(each_post_href_css).get_attribute('href'),every_href_list)) # 현재 페이지 url 스크랩 함수
            # 임시리스트를 전체리스트에 extend
            posts_href_list.extend(posts_href_list_in_page) 

            # URL 추가를 더이상 페이지가 없을 때 까지 진행하는 while 반복문
            # 다음 페이지 이동하기
            Next_page_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[4]/span/span[3]/a'''
            try : # 다음 페이지를 클릭 한다 > xpath가 active 되어있다면
                driver.find_element_by_xpath(Next_page_xpath).click() # 다음페이지로 가서
                element = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'wrap_cont')))
                # 다음 페이지에 기사가 모두 로드 될 때 까지 대기한다
            except : # de-active 되어있다면 > 마무리 하기 
                # 다음 페이지 클릭 함수 > while 반복 
                i = 1 # de-active되어 있다면 >  > while 벗어나기
    # 페이지 바가 없는 경우
    else:
        time.sleep(1)
        posts_href_list_in_page=[] # 임시리스트를 비우고
        # 현재 페이지의 전체 기사 URL을 찾아온다.
        every_posts_info_xpath='''/html/body/article/article/div/article/div/div[2]/div[1]/div[3]'''
        every_posts_info = driver.find_element_by_xpath(every_posts_info_xpath)
        every_href_list = every_posts_info.find_elements_by_css_selector('li') # 기사 하나 당 블록
        each_post_href_css = 'div.wrap_cont > div > div.wrap_tit.mg_tit > a'
        posts_href_list_in_page = list(map(lambda x: x.find_element_by_css_selector(each_post_href_css).get_attribute('href'),every_href_list)) # 현재 페이지 url 스크랩 함수
        # 임시리스트를 전체리스트에 extend
        posts_href_list.extend(posts_href_list_in_page) 

    print('검색어 :',Target_keyword, '// 언론사 : ',journal_target,
          '// 기간 :',period_start_date,'~',period_end_date,'에 대해')
    print('기사', str(len(posts_href_list)), '건의 URL주소를 불러왔습니다.\n')


전체  3 개 weeks      ----->     1 번째 week 

검색어 : 공천 // 언론사 :  조선일보 // 기간 : 2019.12.09 ~ 2019.12.15 에 대해
기사 33 건의 URL주소를 불러왔습니다.

전체  3 개 weeks      ----->     2 번째 week 

검색어 : 공천 // 언론사 :  조선일보 // 기간 : 2019.12.16 ~ 2019.12.22 에 대해
기사 21 건의 URL주소를 불러왔습니다.

전체  3 개 weeks      ----->     3 번째 week 

검색어 : 공천 // 언론사 :  조선일보 // 기간 : 2019.12.23 ~ 2019.12.29 에 대해
기사 28 건의 URL주소를 불러왔습니다.



# *** Code Breakdown

* 위에 코드를 다시 한 단계씩 수행해보면서, 어떻게 명령이 수행되는지 파악해보실 수 있습니다. 

#### 1.2.1 다음 메인 접근 

In [6]:
# 크롬드라이버 import
Chrome_directory = '../Driver/chromedriver' 
driver = webdriver.Chrome(Chrome_directory) # < "셀레니움.웹드라이버.크롬(크롬드라이버경로)" 를 'driver'객체로 지정합니다. 
# driver 지정을 늘 새로 > "driver = webdriver.Chrome" 을 항상 같이 해야 url오픈에 에러가 안남 


"selenium.webdriver.Chrome(크롬드라이버경로 " 즉,   
"셀레니움.웹드라이버.크롬(크롬드라이버경로)" 를 'driver'객체로 지정합니다.   

* 이 객체'driver'를 지정함으로써, 셀레니움.웹드라이버 methods를 계속 사용할 수 있도록 합니다.   
앞으로 계속,   
"driver.엘리먼트 찾기_by엑스패스(원하는엘리먼트의 엑스패스)"   
"driver.엘리먼트 찾기_by_css셀렉터(원하는 엘리먼트의 셀렉터).클릭() "   
이런 형태의 명령문을 수행하면서 원하는 웹 탐색을 진행하게 됩니다. 

In [ ]:
# 크롬드라이버를 통해 Daum News open
driver.get("https://www.daum.net/")
driver.implicitly_wait(10) # 페이지 이동 중에 로드(상태)가 다 될 때까지 최장 10초를 대기

#### 1.2.2) 검색어 입력 및  기간 + 언론사 필터 지정


In [7]:
### 검색어 입력
# SearchBox keyword 입력
# Target_keyword = '20대 국회'  # < 위에서 입력값을 받습니다.
driver.find_element_by_css_selector('input#q.tf_keyword').clear() 
driver.find_element_by_css_selector('input#q.tf_keyword').send_keys(Target_keyword) 

In [8]:
# '검색' 클릭
driver.find_element_by_css_selector('button.ico_pctop.btn_search').send_keys(Keys.ENTER) # '검색'클릭
# 페이지가 넘어가기 때문에 '원하는 작업'을 수행하기위한 wait의 default 값을 지정해줍니다.
wait = WebDriverWait(driver, 60) # 완료되기를 기대하는 '지정된 작업'에 대해 Max 60초를 기다립니다.작업이 완료되지 않으면 Timeout에러가 발생합니다. 

In [9]:
# 페이지가 넘어가기 때문에 '암묵적 대기'를 걸어줍니다.
driver.implicitly_wait(10) # 페이지 이동 중에 로드(상태)가 다 될 때까지 최장 10초를 대기

In [10]:
# 카테고리 > '뉴스' 클릭
# 검색 카테고리는 검색어에 따라 그 위치가 달라집니다. 
# Xpath로 '뉴스'클릭을 지정할 경우 ...div[인덱스숫자].. 에 인덱스 숫자가 3또는 4로 달라지면서
# 다른 카테고리로 유입될 수 있습니다. > 따라서, 카테고리 탭 안에서 현재 '뉴스'탭의 위치를 찾고 
# 그 후 클릭을 수행합니다. 

cat_tab_elements = driver.find_element_by_css_selector('ul.gnb_search').find_elements_by_css_selector('span.txt_tab')
for cat_idx in range(len(cat_tab_elements)):
    if cat_tab_elements[cat_idx].text == '뉴스':
        cat_tab_elements[cat_idx].click()
        break #  참고 * )
# * : '뉴스'탭을 클릭한 후에는, 나머지 탭에 대한 경로가 유효하지 않지만, for는 끝나지 않았기 떄문에
# StaleElementReferenceException 에러가 발생하게 됩니다. break로 반복문이 안전하게 Exit할 수 있도록 지정해줍니다. 

driver.implicitly_wait(10) # 페이지 이동 중에 로드가 다 될 때 까지 최장 10초 대기

In [11]:
### 기간 설정
# 예시로 2주에 해당하는 날짜를 넣었습니다. 12.09.월 ~ 12.22.일
period_start_date = '2018-12-09'
period_end_date = '2018-12-22'

In [12]:
period_more_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[2]/div[1]/a/span[2]'''
try: # 로딩이 채 끝나기전 .click()이 수행되면 에러가 발생합니다. > 따라서 클릭이 가능할 때 까지 wait를 수행합니다.
    element = wait.until(EC.element_to_be_clickable((By.XPATH, period_more_xpath)))
    driver.find_element_by_xpath(period_more_xpath).click() # '기간 더보기 화살표'클릭
except:
    print('기사 검색 기간 선택에 에러 발생')
    print('period_start_date :',period_start_date, 'period_end_date : ',period_start_date)


In [13]:
period_start_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[2]/div[1]/div/div/div[1]/input'''
period_end_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[2]/div[1]/div/div/div[2]/input'''
apply_btn_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[2]/div[1]/div/div/button'''

# 검색 시작 기간 입력
driver.find_element_by_xpath(period_start_xpath).clear() 
driver.find_element_by_xpath(period_start_xpath).send_keys(period_start_date)

# 검색 종료 기간 입력
driver.find_element_by_xpath(period_end_xpath).clear() 
driver.find_element_by_xpath(period_end_xpath).send_keys(period_end_date)

driver.find_element_by_xpath(apply_btn_xpath).send_keys(Keys.ENTER) 
#'적용'버튼에 직접적으로 'enter'키 입력 (for unclickable 해결)

driver.implicitly_wait(10) # 페이지 이동 중에 로드가 다 될 때 까지 최장 10초 대기


### 엘리먼트.click()이 갑자기 안되는 경우   
* 웹 탐색 경로를 하나하나 찾아가면서 코드를 작성할 때는 에러가 안나다가, 전체를 한번에 수행하면 갑자기 에러가 나는 경우가 종종 있습니다.   
* 웹 탐색에 에러를 유발하는 경우가 가장 크게는 두가지로,   
> (1.) 엘리먼트.click()을 하는데 '클릭'이 방해 받는 경우    
> (2.) 클릭등을 수행하고 페이지가 넘어가는데, 새 페이지 로드가 다 끝나기 전에 '엘리먼트 찾기'등의 명령을 수행해서 못 찾는 경우   
>  

등이 있습니다. 

* 위에 '기간선택'을 위해 send_keys를 통해 '2019-12-02'와 같은 값들을 넣어주고, '적용'버튼을 '클릭'하려고 하니, '캘린터 팝업'이 사라지지 않아서 '클릭이 방해받는 에러'가 발생하게 됩니다.   
=> 이런 경우에 대한 해결책은   
> 1. '.click()' 대신 '.send_keys(Keys.ENTER)'명령으로 대체하여, '물리적인 클릭'이 아니라 '그 위치에 엔터 누르기'로 바꿔서 해결해줄 수도 있습니다.   
> *** 이 방법으로 해결이 되지 않는 경우도 있습니다. 다른 해결방법들은 크게 두-세가지 방법들이 있습니다. 아래 링크를 참고해주세요.   (https://stackoverflow.com/questions/52400233/error-other-element-would-receive-the-click-in-python)    

In [14]:
### 언론사 설정 
journal_more_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[2]/div[2]/a/span[2]'''
driver.find_element_by_xpath(journal_more_xpath).click() # '언론사 더보기'클릭

journal_input_xpath='''/html/body/article/article/div/article/div/div[2]/div[1]/div[2]/div[2]/div/ul/li[2]/a'''
driver.find_element_by_xpath(journal_input_xpath).click() # '언론사 직접입력'클릭

journal_target_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[2]/div[2]/div/ul/li[2]/div/div[1]/input'''
journal_apply_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[2]/div[2]/div/ul/li[2]/div/button'''

# 타겟 언론사 입력
driver.find_element_by_xpath(journal_target_xpath).clear() 
driver.find_element_by_xpath(journal_target_xpath).send_keys(journal_target)

driver.find_element_by_xpath(journal_apply_xpath).send_keys(Keys.ENTER)
#'적용'버튼에 직접적으로 'enter'키 입력 (for unclickable 해결)


In [15]:
element = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'wrap_cont')))
time.sleep(1)
# 기사가 다 로드 될 때 까지 최장 60초를 기다린다. 


#### 1.2.3 기사들의 URL 스크랩

In [16]:
### 스크래핑 할 모든 기사들의 URL 주소를 하나의 리스트에 긁어옵니다.
posts_href_list = [] # <<< 전체 url을 담을 리스트
paging_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[4]'''
# 페이지 바가 있는 경우
if (driver.find_element_by_xpath(paging_xpath).get_attribute('class')) == 'paging_comm':
    i = 0 
    while i == 0: 
        time.sleep(1)
        posts_href_list_in_page=[] # 임시리스트를 비우고
        # 현재 페이지의 전체 기사 URL을 찾아온다.
        every_posts_info_xpath='''/html/body/article/article/div/article/div/div[2]/div[1]/div[3]'''
        every_posts_info = driver.find_element_by_xpath(every_posts_info_xpath)
        every_href_list = every_posts_info.find_elements_by_css_selector('li') # 기사 하나 당 블록
        each_post_href_css = 'div.wrap_cont > div > div.wrap_tit.mg_tit > a'
        posts_href_list_in_page = list(map(lambda x: x.find_element_by_css_selector(each_post_href_css).get_attribute('href'),every_href_list)) # 현재 페이지 url 스크랩 함수
        # 임시리스트를 전체리스트에 extend
        posts_href_list.extend(posts_href_list_in_page) 

        # URL 추가를 더이상 페이지가 없을 때 까지 진행하는 while 반복문
        # 다음 페이지 이동하기
        Next_page_xpath = '''/html/body/article/article/div/article/div/div[2]/div[1]/div[4]/span/span[3]/a'''
        try : # 다음 페이지를 클릭 한다 > xpath가 active 되어있다면
            driver.find_element_by_xpath(Next_page_xpath).click() # 다음페이지로 가서
            element = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'wrap_cont')))
            # 다음 페이지에 기사가 모두 로드 될 때 까지 대기한다
        except : # de-active 되어있다면 > 마무리 하기 
            # 다음 페이지 클릭 함수 > while 반복 
            i = 1 # de-active되어 있다면 >  > while 벗어나기
# 페이지 바가 없는 경우
else:
    time.sleep(1)
    posts_href_list_in_page=[] # 임시리스트를 비우고
    # 현재 페이지의 전체 기사 URL을 찾아온다.
    every_posts_info_xpath='''/html/body/article/article/div/article/div/div[2]/div[1]/div[3]'''
    every_posts_info = driver.find_element_by_xpath(every_posts_info_xpath)
    every_href_list = every_posts_info.find_elements_by_css_selector('li') # 기사 하나 당 블록
    each_post_href_css = 'div.wrap_cont > div > div.wrap_tit.mg_tit > a'
    posts_href_list_in_page = list(map(lambda x: x.find_element_by_css_selector(each_post_href_css).get_attribute('href'),every_href_list)) # 현재 페이지 url 스크랩 함수
    # 임시리스트를 전체리스트에 extend
    posts_href_list.extend(posts_href_list_in_page) 
    
print('검색어 :',Target_keyword, '// 언론사 : ',journal_target,
      '// 기간 :',period_start_date,'~',period_end_date,'에 대해')
print('기사', str(len(posts_href_list)), '건의 URL주소를 불러왔습니다.')


검색어 : 공천 // 언론사 :  조선일보 // 기간 : 2018-12-09 ~ 2018-12-22 에 대해
기사 25 건의 URL주소를 불러왔습니다.


In [17]:
posts_href_list
# 이런 형태로 URL을 받아옵니다.

['http://v.media.daum.net/v/20181212190131362?f=o',
 'http://v.media.daum.net/v/20181211030631451?f=o',
 'http://v.media.daum.net/v/20181210102151671?f=o',
 'http://v.media.daum.net/v/20181213233000336?f=o',
 'http://v.media.daum.net/v/20181211031731589?f=o',
 'http://v.media.daum.net/v/20181215030722792?f=o',
 'http://v.media.daum.net/v/20181217030717327?f=o',
 'http://v.media.daum.net/v/20181211175211689?f=o',
 'http://v.media.daum.net/v/20181211030030279?f=o',
 'http://v.media.daum.net/v/20181214111956674?f=o',
 'http://v.media.daum.net/v/20181213030914533?f=o',
 'http://v.media.daum.net/v/20181215173639446?f=o',
 'http://v.media.daum.net/v/20181217030758351?f=o',
 'http://v.media.daum.net/v/20181217030801356?f=o',
 'http://v.media.daum.net/v/20181213160048131?f=o',
 'http://v.media.daum.net/v/20181215030653777?f=o',
 'http://v.media.daum.net/v/20181214032844174?f=o',
 'http://v.media.daum.net/v/20181210030533690?f=o',
 'http://v.media.daum.net/v/20181217030757350?f=o',
 'http://v.m